# Projekt MSP1 / 2024
Cílem tohoto projektu je se seznámit s programovými nástroji využívaných ve statistice a osvojit si základní procedury. Projekt není primárně zaměřen na efektivitu využívání programového vybavení (i když úplně nevhodné konstrukce mohou mít vliv na hodnocení), ale nejvíce nás zajímají vaše statistické závěry a způsob vyhodnocení. Dbejte také na to, že každý graf musí splňovat nějaké podmínky - přehlednost, čitelnost, popisky.

V projektu budete analyzovat časy běhu šesti různých konfigurací algoritmů. Ke každé konfiguraci vzniklo celkem 200 nezávislých běhů, jejichž logy máte k dispozici v souboru [logfiles.zip](logfiles.zip).

Pokud nemáte rozchozené prostředí pro pro spouštění Jupyter notebooku, můžete využití službu [Google Colab](https://colab.google/). Jakákoliv spolupráce, sdílení řešení a podobně je zakázána!

S případnými dotazy se obracejte na Vojtěcha Mrázka (mrazek@fit.vutbr.cz).

__Odevzdání:__ tento soubor (není potřeba aby obsahoval výstupy skriptů) do neděle 27. 10. 2024 v IS VUT. Kontrola bude probíhat na Pythonu 3.12.3 (standardní instalace Ubuntu); neočekává se však to, že byste používali nějaké speciality a nekompatibilní knihovny. V případě nesouladu verzí a podobných problémů budete mít možnost reklamace a prokázání správnosti funkce. Bez vyplnění vašich komentářů a závěrů do označených buněk nebude projekt hodnocen!

__Upozornění:__ nepřidávejte do notebooku další buňky, odpovídejte tam, kam se ptáme (textové komentáře do Markdown buněk)

__Tip:__ před odevzdáním resetujte celý notebook a zkuste jej spustit od začátku. Zamezíte tak chybám krokování a editací, kdy výsledek z buňky na konci použijete na začátku.

__OTÁZKA K DOPLNĚNÍ:__

_Michal Ľaš (xlasmi00)_

## Načtení potřebných knihoven
Načtěte knihovny, které jsou nutné pro zpracování souborů a práci se statistickými funkcemi.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import json
from zipfile import ZipFile

## Načtení dat do DataFrame
Ze souboru `logfiles.zip` umístěném ve stejném adresáři načtěte data a vytvořte Pandas DataFrame.

Výsledky jsou uložené ve formátu JSON - pro zpracování použijte knihovnu `json`.
Můžete využít následující kostru - je vhodné pracovat přímo se ZIP souborem. Jedinou nevýhodou může být to, že vám bude vracet _byte_ objekt, který musíte přes funkci `decode` zpracovat.

Upravte také pomocí funkce `.astype()` datové typy patřičných sloupců.

```py
data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        # TODO test názvu souboru
        with zf.open(filename, "r") as f:
            pass # vytvořte slovník

df = pd.DataFrame(data)
df
```

In [ ]:
data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        if filename.endswith(".json"):
            with zf.open(filename, "r") as f:
                # content = f.read().decode('utf-8')  # byte to string
                logs = json.loads(f.read())
                data.append(logs)

df = pd.DataFrame(data)
df

## Analýza a čištění dat
Vhodným způsobem pro všechny konfigurace analyzujte časy běhů a pokud tam jsou, identifikujte hodnoty, které jsou chybné. Vyberte vhodný graf, který zobrazí samostatně jednotlivé konfigurace.

In [ ]:
# Chybné hodnoty
df_err = df[df['status'] != 'SUCCESS']
print(df_err)

# Zbrazenie všetkých hodnôt pomocou grafu boxplot
order = ['config1', 'config2', 'config3', 'config4', 'config5', 'config6']
labels = ['Conf. 1', 'Conf. 2', 'Conf. 3', 'Conf. 4', 'Conf. 5', 'Conf. 6']

fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(x='configuration', y='runtime', data=df, ax=ax, order=order)
ax.set_xlabel('Configurations')
ax.set_ylabel('Runtime')
ax.set_xticks(range(len(order)))
ax.set_xticklabels(labels)
ax.tick_params(axis='x', labelrotation=45)


__OTÁZKA K DOPLNĚNÍ:__

_Objevily se nějaké chybné hodnoty? Proč tam jsou s ohledem na to, že se jedná o běhy algoritmů? Proč jste zvolili tento typ grafu?_

V každej konfigurácii algoritmu sa objavila nejaká chybná hodnota/y. Buď išlo o chybu *TIME LIMIT*, teda beh algoritmu trval príliš dlho, až pravdepodobne dosiahol stanovený časový limit pre jeden test (1800 časových jednotiek) alebo došlo k chybe *segmentation fault* a algoritmus sa k behu vôbec nedostal, teda čas behu bol 0 časových jednotiek. Vzhľadom k tomu, že sa jedná o algoritmy chyba *TIME LIMIT* môže byť spôsobená napríklad nekonečným cyklom a chyba *segmentation fault* typicky nastáva pri prístupe mimo pridelený pamäťový priestor programu.

Graf *boxplot* som použil, pretože je vhodný pre zobrazenie a porovnanie dát viacerých premenných (v tomto prípade rýchlosti rôznych konfigurácii). Rovnako *boxplot* zobrazuje veľa užitočných parametrov ako: medián, 1. a 3. kvartil a variabilitu dát. V tomto prípade však bolo najviac užitočné, že graf *boxplot* ukazuje odľahlé hodnoty (anglicky outliers), ktoré sa pohybujú výrazne mimo väčšiny ostatných hodnôt. Takéto hodnoty sú z hľadiska merania času behu algoritmov obzvlásť podozrivé a môžu nazančovať chyby merania. Ďalej je možné vidieť, že všetky ostatné parametre sú ťažko čitateľné práve kvôli týmto hodnotám, keďže os *y* musí byť "natiahnutá".

Po vyčistení dát od chybných hodnôt je graf oveľa lepšie čitateľný. Stále sú tu nejaké odľahlé hodnoty, ale už prílišne nevybočujú od ostatných nameraných hodnôt. Z tohto grafu už môžeme ľahko vyčítať všetky parametre, ktoré nám _boxplot_ ako graf poskytuje. Tieto hodnoty nám následne potvrdí deskriptívny popis hodnôt, ktorý je v ďalšej sekcii.

Vyčistěte dataframe `df` tak, aby tam tyto hodnoty nebyly a ukažte znovu analýzu toho, že čištění dat bylo úspěšné. Odtud dále pracujte s vyčištěným datasetem.

In [ ]:
from matplotlib import ticker

df_filtered = df[df['status'] == 'SUCCESS']

fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(x='configuration', y='runtime', data=df_filtered, ax=ax, order= order)
ax.set_xlabel('Configurations')
ax.set_ylabel('Runtime')
ax.set_xticks(range(len(order)))
ax.set_xticklabels(labels)
ax.tick_params(axis='x', labelrotation=45)
ax.yaxis.set_major_locator(ticker.MultipleLocator(10))
ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, color='gray')

## Deskriptivní popis hodnot
Vypište pro jednotlivé konfigurace základní deskriptivní parametry.  

__TIP__ pokud výsledky uložíte jako Pandas DataFrame, zobrazí se v tabulce.

In [ ]:
descriptive_stats = df_filtered.groupby('configuration')['runtime'].describe()
descriptive_stats

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte, co všechno můžeme z parametrů vyčíst._

Tieto parametre nám poskytujú všetky informácie, ktoré sme mohli vidieť v predchádzajúcom _boxplot_ grafe. Navyše tu je počet počet vzoriek pre každú konfiguráciu, priemer a smerodatná odchýlka, ktoré na _boxplot_ grafe vidieť nie sú. Parametre sú v poradí _count_ - počet vzorkov, _mean_ - priemerná hodnota, _std_ - smerodatná odchýlka, _min_ - minimálna hodnota, _25%_ - 1. kvartyl, _50%_ - medián, _75%_ - 3. kvartyl a _max_ - maximálna hodnota.

## Vizualizace
Vizualizujte časy běhů algoritmů tak, aby byl v jednom grafu zřejmý i rozptyl hodnot, avšak bylo možné porovnání. Zvolte vhodný graf, který pak níže komentujte.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

print(df_filtered['runtime'].min())

# Data sú už filtorvané, takže použijem parameter `cut=0`, inak kvôli extrapolácii vo violin grafe v kunfiguráciách 4 a 6 dôjde k prekročeniu do záporných hodnôt
sns.violinplot(x='configuration', y='runtime', data=df_filtered, ax=ax, order=order, cut=0)
ax.set_xlabel('Configurations')
ax.set_ylabel('Runtime')
ax.set_xticks(range(len(order)))
ax.set_xticklabels(labels)
ax.yaxis.set_major_locator(ticker.MultipleLocator(15))
ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.5, color='gray')
ax.tick_params(axis='x', labelrotation=45)

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte  výsledky z tabulky._

Z tabuliek a grafov môžeme pozorovať nasledovné:

1. _Konfigurácia 1_: Pri tejto konfigurácia je najzaujímavejší veľmi malý rozptyl hodnôt. Všetky hodnoty sa husto pohybujú medzi _~74_ - _~127_ časovými jednotkami. Najčastejšie okolo mediánu, čiže _~100_ časovými jednotkami. V tomto ohľadne môžeme konštatovať, že algoritmus je veľmi stabilný. Zároveň by som však podotkol, že táto konfigurácia mala až 7 chybných hodnôt čo ja najviac z pomedzi všetkých konfigurácii. 

2. _Konfigurácie 2_: Táto konfigurácia je celkovo horšia ako konfigurácia 1. Má horší priemerný čas, vyššie položený medián, väčší rozptyl aj rozdiely medzi zaznamenaným minimom a maximom. 

3. _Konfigurácie 3_: Táto konfgurácia je podobná konfigurácii 2. Má o niečo lepšiu priemernú hodnotu, nižšie položený medián a menej významné extrémy. Oproti konfigurácii 1 však stála zaostáva vo všetkých parametroch.

4. _Konfigurácia 4_: Najzaujímavejšou charaktersitikou je obrovský rozptyl hodnôt a výrazné extrémy. Priemer a medián je lepší ako pri konfiguráciách 2 a 3. Algoritmus v tejto konfigurácii sa však správa mimoriadne zvláštne. Za pokus by určite stálo zistiť, kedy táto konfigurácia funguje extrémne rýchlo a kedy naopak pomaly a použiť ju len v špecifických prípadoch. Konfigurácia 1 sa však kvoli lepšiemu priemeru a mediánu zdá v tomto prípade lepšou a stabilnejšou voľbou.

5. _Konfigurácia 5_: Priemerom, mediánom, maximálnou aj minimálnou hodnotou je to najhoršia konfigurácia. Inak tu nevidím žadne pozitíva.

6. _Konfigurácia 6_: Túto konfiguráciu je vhodné porovnať s konfiguráciou 4. Obe konfigurácie majú veľký rozptyl hodnôt, na rozdiel od ostatných. Táto konfigurácia má však lepší priemer, medián a maximálnu hodnotu.

Celkovo mi príde ako najlepšia voľba konfigurácia 1 alebo 6. Konfigurácia 1 je veľmi stabilná voľba, ktorá dosahuje dobré výsledky. Na druhej strane konfigurácia 6 môže byť výrazne rýchlejšia (alebo pomalšia) oproti konfigurácii 1.

Dôležitá vec je aj to, že všetky konfiguráciu majú priemer veľmi blízko svojmu mediánu. Zároveň žiadna konfigurácia nemá hodnoty, ktoré by sa príliš líšili od ostatných. To naznačuje, že sa jedná o normálne rozdelenie.

## Určení efektivity konfigurací algoritmů
Nás ale zajímá, jaká konfigurace je nejrychlejší. Z výše vykresleného grafu můžeme vyloučit některé konfigurace. Existuje tam však minimálně jedna dvojice, u které nedokážeme jednoznačně určit, která je lepší - pokud nebudeme porovnávat pouze extrémní hodnoty, které mohou být dané náhodou, ale celkově. Proto proveďte vhodný test významnosti - v následující části diskutujte zejména rozložení dat (i s odkazem na předchozí buňky, variabilitu vs polohu a podobně). Je nutné každý logický krok a výběry statistických funkcí komentovat. 

Vužijte vhodnou funkci z knihovny `scipy.stats` a funkci poté __implementujte sami__ na základě základních matematických funkcí knihovny `numpy` případně i funkcí pro výpočet vybraného rozložení v [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html). Při vlastní implementaci není nutné se primárně soustředit na efektivitu výpočtu (není potřeba využít všechny funkce numpy, můžete použít normální cykly a podobně - v hodnocení však bude zahrnuta přehlednost a neměly by se objevit jasné chyby, jako je zvýšení třídy složitosti a podobně).

__OTÁZKA K DOPLNĚNÍ:__

_Jaká data budete zkoumat? Jaké mají rozložení a parametry (např. varianci) a jaký test použijete? Jaká je nulová hypotéza? Jak se liší variabilita a poloha vybraných konfigurací?_

Osobne tu vidím dva páry konfigurácii, pri ktorých nedokážem určiť, ktorá je lepšia. Konkrétne sú to konfigurácie 2 a 3, alebo 1 a 6. Predpokladám však, že nás zaujíma najlepšia konfigurácia. Preto budem ďalej pracovať s konfiguráciami 1 a 6, keďže tie, ako som spomenul, považujem za najlepšie.

_Konfigurácia 1_ má o trochu lepší priemer (~99.59) a medián (~99.87). Najpodstatnejšie však je, že má veľmi malý rozptyl (~9.39). _Konfigurácia 6_ má nopak veľmi veľký rozptyl (~40.88). V niektorých nameraných hodnotých výrazne prekonáva konfiguráciu 1 a v iných je naopak výrazne pomalšia. Priemer hodnôt je 110.42 a medián je 111.04. Rozdiely medzi konfiguráciami 1 a 6 popisuje nasledujúca tabuľka:

|   | priemer | medián | rozptyl | min | max |
|---|---|---|---|---|---|
| _Rozdiel_ | 10.832275 | 11.178614 | 31.488966 | 71.678015 | 88.093591 |

Chceme zistiť, ktorá konfigurácia je na tom priemerne lepšie (čiže budem skúmať priemer). Budem predpokladať, že sa jedná o normálne rozdelenia, čo aj overím testom normality (Shapiro-Wilk test). Následne použijem jednostranný _T-test_ pre dve nezávislé vzorky na porovnanie priemerov. T-test umožňuje zistiť, či je rozdiel medzi priemermi týchto dvoch konfigurácií štatisticky významný. Treba však dbať na to, že klasický T-test predpokladá, že rozptyly dvoch vzoriek sú približne rovnaké. Preto použijem Welchov T-test, ktorý sa používa pri rozdielnych hodnotách rozptylu.

Nulová hypotéza znie: _Medzi priemernými časmi konfigurácie 1 a konfigurácie 6 nie je žiadny štatisticky významný rozdiel._

Alternatívna hypotéza​: _Konfigurácia 1 má menší priemerný čas behu ako konfigurácia 6._

Ak by platila nulová hypotéza, nevieme určiť, ktorá konfigurácia je lepšia, pretože medzi priemernými časmi behu nie je žiadny štatisticky významný rozdiel. Naopak, ak by platila alternatívna hypotéza, tak by konfigurácia 1 bola lepšie, pretože má lepší priemerný čas.

In [ ]:
# Výber dát pre config1 a config6
config1_data = df_filtered[df_filtered['configuration'] == 'config1']['runtime']
config6_data = df_filtered[df_filtered['configuration'] == 'config6']['runtime']

# Test normality (Shapiro-Wilk test)
def test_normal_dist(x) -> tuple[float, bool]:
    _, p = stats.shapiro(x)
    if p < 0.05:
        return (p, False)
    else:
        return (p, True)

config1_normal = test_normal_dist(config1_data)
config6_normal = test_normal_dist(config6_data)

print(f"Konfigurácia 1: p-hodnota normality {config1_normal[0]}, dáta {'majú' if config1_normal[1] else 'nemajú'} normálne rozloženie.")
print(f"Konfigurácia 6: p-hodnota normality {config6_normal[0]}, dáta {'majú' if config6_normal[1] else 'nemajú'} normálne rozloženie.")


# Jednostranný Welchov T-test (nevyžaduje rovnaké rozptyly, záleží na poradí parametrov)
_, p_welch = stats.ttest_ind(config1_data, config6_data, equal_var=False, alternative='less', nan_policy='raise')
print(f"Welchov T-test konfigurácie 1 oproti konfigurácii 6: p-hodnota {p_welch}")


__OTÁZKA K DOPLNĚNÍ:__

_Jaký je závěr statistického testu?_

Výsledok T-testu: _Welchov T-test konfigurácie 1 oproti konfigurácii 6: p-hodnota 0.00016959241379424815_

Hodnota _p_ Walchovho T-testu vyšla menšia ako _0.05_, teda zamietame nulovú hypotézu a potvrdzujeme alternatívnu hypotézu.

Výsledok je, že konfigurácia 1 má s 95-percentnou pravdepodobnosťou menšiu priemernú dobu behu ako konfigurácia 6.

### Vlastní implementace
Implementujte stejný test pomocí knihovních funkcí a ukažte, že je výsledek stejný.

In [ ]:
from scipy.stats import t

# implementácia bolo vytvorená pomocou vzorcov z: https://en.wikipedia.org/wiki/Student%27s_t-test#Independent_two-sample_t-test

def my_ttest_ind(a, b, equal_var=True, alternative='two-sided'):

    # Počet dát
    n1 = len(a)
    n2 = len(b)

    # Priemery
    x1 = np.mean(a) # priemer a
    x2 = np.mean(b) # priemer b

    # Rozptyly (ddof=1 je Besselova korekcia pre výberový rozptyl)
    var1 = np.var(a, ddof=1) # s1^2
    var2 = np.var(b, ddof=1) # s2^2

    # Ak predpokladáme rovnaké rozptyly
    if equal_var:
        pooled_var = ((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2)
        t_stat = (x1 - x2) / np.sqrt(pooled_var * (1 / n1 + 1 / n2))
        df = n1 + n2 - 2
    # Ak predpokladáme rozdielne rozptyly (equal_var=False)
    else:    
        t_stat = (x1 - x2) / np.sqrt(var1 / n1 + var2 / n2)
        df = ((var1 / n1 + var2 / n2) ** 2) / (((var1 / n1) ** 2 / (n1 - 1)) + ((var2 / n2) ** 2 / (n2 - 1)))
 
    if alternative == 'two-sided':
        p_value = 2 * t.sf(np.abs(t_stat), df)
    elif alternative == 'greater':
        p_value = 1 - t.cdf(t_stat, df)
    elif alternative == 'less':
        p_value = t.cdf(t_stat, df)
    else:
        raise ValueError("Neznáma hodnota parametru 'alternative'. Vyber z možností 'two-sided', 'greater' alebo 'less'.")
    
    return t_stat, p_value


# Použitie
_, p_value_custom = my_ttest_ind(config1_data, config6_data, equal_var=False, alternative='less')
print(f"Vlastný T-test: p-hodnota = {p_value_custom}")

# Porovnanie s funkciou scipy.stats.ttest_ind
_, p_value_scipy = stats.ttest_ind(config1_data, config6_data, equal_var=False, alternative='less', nan_policy='raise')
print(f"Scipy T-test: p-hodnota = {p_value_scipy}")
